In [ ]:
"""
sample code of
using sklearn.linear_model.LogisticRegression
"""

import numpy as np
X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 1, 2])

print X.shape
print y.shape

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)
#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, \
#    decision_function_shape=None, degree=3, gamma='auto', \
#    kernel='rbf', max_iter=-1, probability=False, random_state=None, \
#    shrinking=True, tol=0.001, verbose=False)
print(clf.predict_proba([[3, 4], [1, 1]]))


In [ ]:
# plow multiple images

"""
plot image matrix
"""

# plot 3x3 picture matrix

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import requests
import StringIO
from matplotlib.pyplot import figure, show, axes, sci
from PIL import Image


def thumbnail(img):
    """ generate 250x250 square thumbnail """
    
    THUMB_SIZE = 250, 250
    width, height = img.size

    if width > height:
        delta = width - height
        left = int(delta/2)
        upper = 0
        right = height + left
        lower = height
    else:
        delta = height - width
        left = 0
        upper = int(delta/2)
        right = width
        lower = width + upper

    img = img.crop((left, upper, right, lower))
    img.thumbnail(THUMB_SIZE, Image.ANTIALIAS)
    
    return img


def plot_3x3(images, title = ''):
    """
    given 9 images, plot them in 3x3 matrix
    """

    plt.figure(figsize=(10,10))
    
    Nr = 3
    Nc = 3
    i = 0
    
    for image in images:
        i += 1
        img_io = requests.get(image)
        plt.subplot(Nr, Nc, i)
        
        image = Image.open(StringIO.StringIO(img_io.content))
        image = thumbnail(image)
        
        #img = io.imread(StringIO.StringIO(img_io.content))
        plt.imshow(image)
    
    plt.show()


In [ ]:
"""
load positive and negative examples
"""

import os

print 'Loading positive samples...'

filepath = '../tasks/03.image-crawl'
filename = 'result-wse_flickr.jsonl'

filename = os.path.join(filepath, filename)

# load the data
# only get the data for query='bench'

lines = open(filename).readlines()

import json
items = [json.loads(line) for line in lines]
items = filter(lambda item: item.get('embeds') and item.get('query') == 'bird', items)

urls = [item['url'] for item in items]
features = [item['embeds'] for item in items]


#
# negative data
#
print 'Loading negative samples...'

filename_neg = 'result-wse_non_samples.jsonl'
filename_neg = os.path.join(filepath, filename_neg)

lines = open(filename_neg).readlines()
lines = lines[:1000]

items = [json.loads(line) for line in lines]
items = filter(lambda item: item.get('embeds') and len(item.get('embeds')) == 1024 and item.get('query') != 'bird', items)

urls_neg = [item['url'] for item in items]
features_neg = [item['embeds'] for item in items]


In [ ]:
#
# balance the data with top 100 results
#
urls_pos = urls[:100]
features_pos = features[:100]

urls_neg = urls_neg[:100]
features_neg = features_neg[:100]

print 'sample checking'
print '%d positive samples' % len(features_pos)
print '%d negative samples' % len(features_neg)

urls_test = urls[100:300]
features_test = features[100:300]
print '%d test samples' % len(urls_test)

print urls_pos[1]
print features_pos[1][:10]

print urls_neg[1]
print features_neg[1][:10]

print urls_test[1]
print features_test[1][:10]


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

train_data = np.array(features_pos + features_neg)
train_label = np.array([1] * 100 + [2] * 100)

print train_data.shape
print train_label.shape

print len(train_data)
print len(train_label)

assert(len(train_data) == len(train_label))

clf = LogisticRegression()
clf.fit(train_data, train_label)


In [ ]:
print 'test positive samples:'
for test_feature in features_test[:20]:
    print(clf.predict_proba(test_feature))
plot_3x3(urls_test[:9])


print 'test negative samples:'
for test_feature in features_neg[-20:]:
    print(clf.predict_proba(test_feature))
plot_3x3(urls_neg[-9:])
